In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime
import subprocess
import os
import sys
sys.path.append('/data100t1/home/wanying/lab_code/utils/')
import CHARGE
print('Last run:', datetime.datetime.now().strftime('%Y-%m-%d'))

Last run: 2024-01-22


# 1. File preparation and cleaning

## 1.1 Phenotypes and data adjustments
Variables to look for in the CCHC data:
1. GHB: HbA1c in % (fasting does not affect this value)
2. MFBG: fasting glucose (mg/dL)

Exclusion criteria:
1. Exclude dibetic subjects
2. Exclude missing values
3. Exclude individuals reporting a total sleep time of less than 3 hours or more than 14 hours
4. Exclude age<18

In [2]:
print('# Load phenotype file')
cols_to_use = ['RRID', 'LABID',  'VISIT', 'AGE_AT_VISIT', 'GENDER', 'ADA2010_DM', 'MFBG', 'GHB', 'psqi_hrsleep'] 
fn_pheno = '/data100t1/share/CCHC/phenotypes/0723/cchc_phenotypes_0723.txt'
df_pheno = pd.read_csv(fn_pheno, sep='|', dtype='str')[cols_to_use]
for col in cols_to_use[2:]:
    df_pheno[col] = pd.to_numeric(df_pheno[col])

# Drop missing values etc.
n_samples = len(df_pheno)
df_pheno = df_pheno[(df_pheno['AGE_AT_VISIT']>=18) & (df_pheno['ADA2010_DM']==0)].copy()
print('# - Exclude subjects (visits) with diabetes or younger than 18 yrs: N dropped =', n_samples-len(df_pheno))
n_samples = len(df_pheno)

df_pheno.dropna(subset=cols_to_use[3:], inplace=True)
print('# - Drop visits with missing values in age, sex, ADA2010_DM, GHB and sleep: N dropped =', n_samples-len(df_pheno))
n_samples = len(df_pheno)

df_pheno = df_pheno[(df_pheno['psqi_hrsleep']>=3) & (df_pheno['psqi_hrsleep']<=14)]
print('# - Drop individuals with sleep time < 3hrs or > 14hrs: N dropped =', n_samples-len(df_pheno))
n_samples = len(df_pheno)

df_pheno.drop_duplicates(subset='RRID', inplace=True)
print('# - Drop duplicate RRID: N dropped =', n_samples-len(df_pheno))

# Merge with PCA (Use PC 1-5)
pca_fn = '/data100t1/home/wanying/CCHC/doc/genotype_and_pc/202210_CCHC_PCs_id_fixed_1KG_controls_removed.txt'
df_pca = pd.read_csv(pca_fn , sep='\t')
pca_cols = ['RRID', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']
df_pheno = df_pheno.merge(df_pca[pca_cols], on='RRID')
print('# - Merge with PC 1-5')

print('# Number of samples remaining:', len(df_pheno))
print('# DONE')
display(df_pheno.head(2))

# Load phenotype file
# - Exclude subjects (visits) with diabetes or younger than 18 yrs: N dropped = 3968
# - Drop visits with missing values in age, sex, ADA2010_DM, GHB and sleep: N dropped = 6156
# - Drop individuals with sleep time < 3hrs or > 14hrs: N dropped = 2
# - Drop duplicate RRID: N dropped = 0
# - Merge with PC 1-5
# Number of samples remaining: 353
# DONE


,RRID,LABID,VISIT,AGE_AT_VISIT,GENDER,ADA2010_DM,MFBG,GHB,psqi_hrsleep,PC1,PC2,PC3,PC4,PC5
0,BD0009,15Y0224,4.0,61.2,1.0,0.0,85.0,5.1,5.0,0.010911,-0.075311,0.006324,0.020142,-0.009451
1,BD0013,15Y0242,14.0,59.8,1.0,0.0,108.0,5.5,8.0,-0.036728,0.014562,0.032178,0.003866,0.005183


## 1.2 Lifestyle variable

“SHORT TOTAL SLEEP TIME” (STST) AND “LONG TOTAL SLEEP TIME” (LTST)
```
Before defining exposure, we request all cohorts (see note and R code below) to regress the total sleep duration on age, sex, and age*sex interaction in the sexes-combined sample, and use those age-sex-adjusted residuals in the combined sample to define the exposure variables in males, females, and the combined sample. You may do this using the R script below. Please exclude all individuals without genetics data prior to calculating the age-sex-adjusted residuals for total sleep duration.
```

### 1.2.1 Drop samples without genotype data

In [3]:
print('# Exclude individuals without genotype data: ', end='')
fn_id_mapping = '/data100t1/home/wanying/CCHC/doc/samples_IDs/20220916_IDs_genotyped_all.txt'
df_gp_id_mapping = pd.read_csv(fn_id_mapping, sep='\t')
n_samples = len(df_pheno)
df_pheno = df_gp_id_mapping.merge(df_pheno, on='RRID')
print('N sample dropped = ', n_samples - len(df_pheno))
print('# Number of samples remaining:', len(df_pheno))

# Rename columns, calcualte sex*age
df_pheno.rename(columns = {'AGE_AT_VISIT':'age', 'GENDER':'sex', 'psqi_hrsleep':'TST'}, inplace=True)

# Code male=0, female=1
df_pheno['sex'] = df_pheno['sex'].replace(1, 0)
df_pheno['sex'] = df_pheno['sex'].replace(2, 1)

# Create age*sex column. Optional
df_pheno['age*sex'] = df_pheno['age'] * df_pheno['sex']
df_pheno.head(2)

# Exclude individuals without genotype data: N sample dropped =  0
# Number of samples remaining: 353


,RRID,genotype_ID,LABID,VISIT,age,sex,ADA2010_DM,MFBG,GHB,TST,PC1,PC2,PC3,PC4,PC5,age*sex
0,BD0009,BD0009_BD4009,15Y0224,4.0,61.2,0.0,0.0,85.0,5.1,5.0,0.010911,-0.075311,0.006324,0.020142,-0.009451,0.0
1,BD0013,BD0013_BD4013,15Y0242,14.0,59.8,0.0,0.0,108.0,5.5,8.0,-0.036728,0.014562,0.032178,0.003866,0.005183,0.0


### 1.2.2 Regress the total sleep duration to get age-sex-adjusted residuals
R is not available on vgipiper06, use other server to run (1) R section below.

Otherwise use the (2) python function in CHARGE

#### (1) R version to get residuals (code from analysis plan)

In [17]:
# %%bash
# pip install rpy2

In [18]:
import rpy2
# Check here for rpy2 extension: 
# https://rpy2.github.io/doc/latest/html/interactive.html#module-rpy2.ipython.rmagic
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [19]:
%%R  -i df_pheno -o df_pheno
# input df_indivs_with_sleep
# Output dataframe with residuals
# Below R code is from analysis plan doc

# df = read.table(”yourPhenotypeFile.txt”, h=T) # read data with headers
# Note this data set should have column with TST, age and sex (men = 0, women = 1).


#Where: 
#TST = total sleep time (continuously in hours)
#age = age of the participant at the time of the sleep assessment
#sex = sex of the participant
#age*sex = the interaction between the age and sex of the participant

#Define exposed participants to Short Total Sleep Time (STST) and Long Total Sleep Time (LTST):

#Example script for calculating the total sleep time residuals.

#For total population:
TST.lm.all = lm(TST ~ age + sex + age*sex, data = df_pheno)
df_pheno$TST.res.all = resid(TST.lm.all)  # age and sex-adjusted (including interaction between these) residuals in all study participants.

quantile(df_pheno$TST.res.all, c(0.20, 0.80)) # 20th (residuals below this threshold will be classified as STST) and 80th (residuals above this threshold will be classified as LTST) percentiles. For the sex-stratified analyses, you can simply make the selection for either men or women; the same residuals can be used.  



       20%        80% 
-0.9879649  1.1548211 


#### (2) Python version to get residuals

In [4]:
# Or use python code if rpy2 does not work (ie. on server vgipiper06)
df_pheno, q2, q8 = CHARGE.get_residuals(df_pheno,
                                        exposure_col='TST',
                                        covar_cols=['sex', 'age', 'age*sex'])

# Set exposure E
# 1. Short Total Sleep Time (STST):
#    STST = 1 if TST.res ≤ 20th percentile (i.e., E = 1)
#    STST = 0 otherwise (E = 0)
# 2. Long Total Sleep Time (LTST):
#    LTST = 1 if TST.res ≥ 80th percentile (i.e., E = 1)
#    LTST = 0 otherwise (E = 0)

df_pheno['STST'] = 0
df_pheno.loc[df_pheno['TST_residuals']<=q2, 'STST'] = 1
df_pheno['LTST'] = 0
df_pheno.loc[df_pheno['TST_residuals']>=q8, 'LTST'] = 1

# Calcualte age*E, age2*E and sex*E
df_pheno['age2'] = df_pheno['age']**2
for exposure in ['LTST', 'STST']:
    df_pheno[f'age*{exposure}'] = df_pheno['age'] * df_pheno[exposure]
    df_pheno[f'age2*{exposure}'] = df_pheno['age2'] * df_pheno[exposure]
    df_pheno[f'sex*{exposure}'] = df_pheno['sex'] * df_pheno[exposure]

# Save output
print('# Save cleaned phenotype and covariates file')
output_dir = '/data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/supporting_files'
output_fn = 'phenotype_and_covars.csv'
if not os.path.isfile(os.path.join(output_dir, output_fn)):
    df_pheno.to_csv(os.path.join(output_dir, output_fn), index=False)
else:
    print('# File already exist. Skip svaing: %s' % output_fn)

# Get residuals of TST
# - 20th percentile: -0.9869025363003179
# - 80th percentile: 1.1573809964079347
# Save cleaned phenotype and covariates file


### 1.2.3 Adjust traits for relatedness, then perform winsorization

#### (1) Run null model to adjust phenotype for relatedness
Create files to run null model

In [ ]:
# ######### Unnecessary #########
# # Create a list of samples to create grm by plink and be used in GWAS and
# fid = df_pheno['genotype_ID'].apply(lambda x: x.split('_')[0])
# iid = df_pheno['genotype_ID'].apply(lambda x: x.split('_')[-1])
# df_sample_list = pd.DataFrame({'FID':fid, 'IID':iid})

# print('# Save list of samples to be used by plink (FID and IID)')
# output_dir = '/data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/supporting_files'
# output_fn = 'sample.list'
# if not os.path.isfile(os.path.join(output_dir, output_fn)):
#     df_sample_list.to_csv(os.path.join(output_dir, output_fn), sep='\t', index=False)
# else:
#     print('# File already exist. Skip svaing: %s' % output_fn)

# Save list of samples to be used by plink (FID and IID)


In [16]:
# # Cread pedigree file for GRM calculate
# '''
# Set value to 0 if missing:
#   PED  - Pedigree ID
#   EGO  - Individual ID
#   FA   - Father ID
#   MO   - Mother ID
#   SEX  - sex (1 = male, 2 = female)
# '''
# df_ped = df_pheno[['genotype_ID', 'sex']].copy()
# df_ped['PED'], df_ped['FA'], df_ped['MO'] = 0, 0, 0
# df_ped['sex'] = df_ped['sex'].replace(1,2).replace(0,1).astype(int)
# df_ped.rename(columns={'genotype_ID':'EGO', 'sex':'SEX'}, inplace=True)
# df_ped = df_ped[['PED', 'EGO', 'FA', 'MO', 'SEX']]

# print('# Save list of samples to be used by plink (FID and IID)')
# output_dir = '/data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/supporting_files'
# output_fn = 'ped.csv'
# if not os.path.isfile(os.path.join(output_dir, output_fn)):
#     df_ped.to_csv(os.path.join(output_dir, output_fn), sep='\t', index=False)
# else:
#     print('# File already exist. Skip svaing: %s' % output_fn)

# Save list of samples to be used by plink (FID and IID)


#### (2) Merge residuals from null model with other covariates and save to file
1. Use files *.poly.model.csv
2. Merge columns EGO and (trait)_ERROR_RESIDUAL to the phenotyoe file.

In [5]:
# Merge null model residuals
null_model_result_dir = '/belowshare/vumcshare/data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/supporting_files/null_model_residuals'
traits = ['GHB', 'MFBG']
exposure = 'sleep'
output_dir = '/belowshare/vumcshare/data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/supporting_files/phenotype_files/'
for fn in os.listdir(null_model_result_dir):
    if fn.endswith('.poly.model.csv'):
        print('\n# Process residuals from null model:', fn)
        df = pd.read_csv(os.path.join(null_model_result_dir, fn))
        for trait in traits:
            res_col = trait+'_ERROR_RESIDUAL'
            if res_col in df.columns:
                print('# Process trait:', trait)
                # Merge residual and phenotype by genotyope ID
                df_merged = df[['EGO', res_col]].merge(df_pheno, left_on='EGO', right_on='genotype_ID')
                # winsorization
                num_winsorized, df_merged[res_col+'_winsorized'] = CHARGE.winsorization(df_merged[res_col])
                
                # Stratify by sex, save each output (male=0, female=1)
                mask_male, mask_female = df_merged['sex']==0, df_merged['sex']==1
                # Combined
                df_merged.to_csv(output_dir+f'{trait}.{exposure}.combined.pheno.csv', index=False)
                # Male
                df_merged[mask_male].to_csv(output_dir+f'{trait}.{exposure}.male.pheno.csv', index=False)
                # Female
                df_merged[mask_female].to_csv(output_dir+f'{trait}.{exposure}.female.pheno.csv', index=False)
                print('# - N samples combined:', len(df_merged))
                print('# - N samples male:', len(df_merged[mask_male]))
                print('# - N samples female:', len(df_merged[mask_female]))
                
print('# DONE')



# Process residuals from null model: GHB.grm.poly.model.csv
# Process trait: GHB
# Winsorize given values
# - Number of values winsorized: 0
# - N samples combined: 353
# - N samples male: 117
# - N samples female: 236

# Process residuals from null model: MFBG.grm.poly.model.csv
# Process trait: MFBG
# Winsorize given values
# - Number of values winsorized: 0
# - N samples combined: 353
# - N samples male: 117
# - N samples female: 236
# DONE


# 2. GWAS

## 2.0 (SKIP) Convert VCF to MMAP binary format

In [2]:
# %%bash
## Example:
# bash 03_SKIP_VCF2MMAP.sh /vgipiper04/CCHC/TOPMed_postimpute_042022/chr22.dose.vcf.gz 22

In [ ]:
# trait = 'glycemia'

# for chr_num in range(1, 23):
#     vcf_fn = f'/vgipiper04/CCHC/TOPMed_postimpute_042022/chr{i}.dose.vcf.gz'
#     cmd = f'screen -dmS {trait}_chr{chr_num}; screen -S {trait}_chr{chr_num} -X stuff '
#     plink_cmd = f"plink2 --vcf {vcf_fn} 'dosage'=DS --out CCHC_chr{chr_num};exit\\n"
#     cmd = f'{cmd} "{plink_cmd}"'
#     print(cmd)

## 2.1 Model 1
Joint analysis of main and interaction effects

In [ ]:
# %%bash
# Example run:
# bash /data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/code/05_GEM_GWAS_model1.sh \
# 22 GHB combined LTST \
# /data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/supporting_files/phenotype_files/GHB.sleep.combined.pheno.csv

In [1]:
# Save commands to file to run
cmd_prefix = 'bash /data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/code/05_GEM_GWAS_model1_and_2.sh'
for trait in ['MFBG', 'GHB']:
    for exposure in ['LTST', 'STST']:
        output_cmd_fn = f'./GWAS_cmds/cmd_GEM_GWAS_{trait}_{exposure}_model1.sh'
        out_fh = open(output_cmd_fn, 'w')
        for sex in ['combined', 'male', 'female']:
            pheno_fn = f'/data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/supporting_files/phenotype_files/{trait}.sleep.combined.pheno.csv'
            for chr_num in range(1, 23):
                cmd = f'{cmd_prefix} {str(chr_num)} {trait} {sex} {exposure} {pheno_fn}'
                out_fh.write(cmd + '\n')
        out_fh.close()

## 2.2 Model 2
Analysis of main effect only

In [ ]:
# %%bash
# Example run:
# bash /data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/code/05_GEM_GWAS_model1.sh \
# 22 GHB combined none \
# /data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/supporting_files/phenotype_files/GHB.sleep.combined.pheno.csv

In [2]:
# Save commands to file to run
cmd_prefix = 'bash /data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/code/05_GEM_GWAS_model1_and_2.sh'
for trait in ['MFBG', 'GHB']:
    for exposure in ['none']:
        output_cmd_fn = f'./GWAS_cmds/cmd_GEM_GWAS_{trait}_{exposure}_model2.sh'
        out_fh = open(output_cmd_fn, 'w')
        for sex in ['combined', 'male', 'female']:
            pheno_fn = f'/data100t1/home/wanying/CCHC/CHARGE_GWAS/202311_Heming_sleep_glycemia/supporting_files/phenotype_files/{trait}.sleep.combined.pheno.csv'
            for chr_num in range(1, 23):
                cmd = f'{cmd_prefix} {str(chr_num)} {trait} {sex} {exposure} {pheno_fn}'
                out_fh.write(cmd + '\n')
        out_fh.close()

## 2.3 Merge results